## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [1]:
from arcgis.gis import GIS
import pandas as pd
import json
# gis = GIS("home")

agol_username = input("Username: ")
agol_password = input("Password: ")

url = 'https://governmentofbc.maps.arcgis.com' # change to your url, whether maphub, geohub, or other
gis = GIS(url, agol_username, agol_password)

#### Now you are ready to start!

In [2]:
# Change to grabbing/updating urls


class map_id_update:
    def __init__(self,primary_id, rest_dict={}, id_dict={}):
        self.primary_id = primary_id
        self.rest_dict = rest_dict
        self.id_dict = id_dict
        self.change_list = []
        self.prim_wm_item = gis.content.get(self.primary_id)
        self.prim_wm_json = self.prim_wm_item.get_data()
    def changes(self, layer_level="initial"): # recursive changes to layers
        if layer_level == "initial":
            layer_level=self.prim_wm_json['operationalLayers']
        for i in reversed(range(len(layer_level))):
            if layer_level[i]['layerType'] == "ArcGISFeatureLayer" and "itemId" in list(layer_level[i].keys()) and layer_level[i]['itemId'] in list(self.id_dict.keys()):
                self.change_list.append(layer_level[i]['itemId'] + " replacement: " + self.id_dict[layer_level[i]['itemId']])
                layer_level[i]['itemId'] = self.id_dict[layer_level[i]['itemId']]
            if layer_level[i]['layerType'] == "ArcGISFeatureLayer" and "url" in (layer_level[i].keys()) and layer_level[i]['url'] in list(self.rest_dict.keys()):
                self.change_list.append(layer_level[i]['url'] + " replacement: " + self.rest_dict[layer_level[i]['url']])
                layer_level[i]['url'] = self.rest_dict[layer_level[i]['url']]
            else:
                if layer_level[i]['layerType'] == 'GroupLayer':
                    self.changes(layer_level[i]['layers'])
    def push(self): # push changes to map
        self.deriv_wm_item = gis.content.get(self.primary_id)
        self.deriv_wm_item.update(data = self.prim_wm_json)


In [3]:
# For populating the dict, use the pandas example that grabbed REST URLs
# grab the csv off github and parse accordingly
# Read csv file from GITHub using pandas
url = 'https://raw.githubusercontent.com/isaac1345/sc_layer_updater/main/old_new_urls.csv'
df = pd.read_csv(url,index_col=0,encoding='cp1252')

id_dict = {}
rest_dict = {}
for index, row in df.iterrows():
    id_dict[row[3]] = row[1]
    rest_dict[row[4]] = row[8]


In [4]:
item_list = []
for item in gis.content.search(query="* AND \  owner:" + gis.users.me.username, max_items=5000):
    item_list.append([item.title,item.id])

In [ ]:
maps = 0
changes_reqd = 0
for in_id in item_list:
    in_id = in_id[1]
#     print(in_id)
    item_prefetch = gis.content.get(in_id)
#     print(item_prefetch)
    if item_prefetch.type == "Web Map":
        map_2 = map_id_update(in_id, rest_dict, id_dict)
        filename = f"{map_2.prim_wm_item.title}_{in_id}.json".replace(":", "-").replace('"', '').replace("|", "_").replace("/", "_").replace("\\", "_")
        filename = f"AGO_JSON_Backups/{filename}"
        with open(filename, "w") as file_handle:
            file_handle.write(json.dumps(map_2.prim_wm_json))
        map_2.changes()
        if len(map_2.change_list) > 0:
            changes_reqd += 1
            print("\n")
            print(f"        {map_2.prim_wm_item.title}")
            for line in map_2.change_list:
                print(line)
            choice = input(f"        {map_2.prim_wm_item.title}:\nMake the above changes? (y/n)\n")
            if choice == "y":
                print("making changes")
                map_2.push()
            else:
                print("Skipped. JSON was still backed up")
        maps+=1
print(f"maps checked: {maps}")
print(f"maps requiring change: {changes_reqd}")